In [6]:
import numpy as np
import matplotlib.pyplot as plt
import os

font={'family':'serif',
      # 'style':'italic',  # 斜体
      'weight':'normal',
      # 'color':'red',
      'size': 18
}  
def setfigform_simple(xlabel, ylabel=None, xlimit = (None,None), ylimit = (None, None), xticks=None):
    # plt.legend(fontsize = 16, frameon=False),
    plt.xlabel(xlabel, fontdict = font)
    plt.ylabel(ylabel, fontdict = font)
    plt.xlim(xlimit)
    plt.ylim(ylimit)
    if xticks is None:
        plt.xticks(fontsize = font['size'], fontname = "serif")
    else:
        plt.xticks(xticks, fontsize = font['size'], fontname = "serif")
    plt.yticks(fontsize = font['size'], fontname = "serif")
    plt.tick_params(direction="in")
    
line_color = [plt.colormaps["brg"](float(i)/float(3)) for i in range(3)]

In [7]:

def ReadReferenceF(filename):
    ofile_prob_E = open(filename,"r")
    Reference_dict = {}
    idx_jj = 0
    while(True):
        line = ofile_prob_E.readline()
        if not line:
            break

        line = ofile_prob_E.readline()
        bin_centers = np.array([float(x) for x in line.split()])

        line = ofile_prob_E.readline()
        jj = float(line.split()[-1].replace("kBT=",""))

        line = ofile_prob_E.readline()
        F = np.array([float(x) for x in line.split()])

        Reference_dict[jj]=np.stack([bin_centers, F])
    ofile_prob_E.close()
    return Reference_dict




In [8]:
def ReadReferenceF_potE(filename, readerror=False):
    ofile_prob_E = open(filename,"r")
    Reference_dict = {}
    idx_jj = 0
    while(True):
        line = ofile_prob_E.readline()
        if not line:
            break

        line = ofile_prob_E.readline()
        bin_centers = np.array([float(x) for x in line.split()])

        line = ofile_prob_E.readline()
        jj = float(line.split()[-1].replace("kBT=",""))

        line = ofile_prob_E.readline()
        F = np.array([float(x) for x in line.split()])
        if readerror:
            line = ofile_prob_E.readline()
            if not "ERROR" in line: 
                raise Exception("ERROR:: ERROR data not found in ", filename)
            line = ofile_prob_E.readline()
            errF = np.array([float(x) for x in line.split()])
        if readerror:
            Reference_dict[jj]=np.stack([bin_centers, F, errF])
        else:
            Reference_dict[jj]=np.stack([bin_centers, F])
    ofile_prob_E.close()
    return Reference_dict



In [9]:
ref_dirname = "/nfs/scistore14/chenggrp/ptuo/NeuralRG/data/ising-latt%dx%d-T4.0/latt%dx%d/"%(6,6,6,6)
Reference_dict_latt6 = ReadReferenceF(os.path.join(ref_dirname, "F-MAGN-REF.dat"))
Reference_dict_potE_latt6 = ReadReferenceF_potE(os.path.join(ref_dirname, "F-E-REF.dat"))


In [10]:
Expectation_Reference_dict_latt6 = {}
Expectation_Reference_dict_latt6["PotE"] = np.loadtxt(os.path.join(ref_dirname, "Expectation-E-REF.dat"), skiprows=1)

In [11]:
print(Expectation_Reference_dict_latt6["PotE"][:,0])
print(Expectation_Reference_dict_latt6["PotE"][:,1]/6**2)

[1.  1.2 1.4 1.6 1.8 2.  2.2 2.4 2.8 3.2 3.6 4.  4.4 4.8 5.2 5.6 6.  8. ]
[-1.97026988 -1.96190508 -1.94176851 -1.90239366 -1.83474054 -1.72269063
 -1.56182895 -1.36432234 -1.00353936 -0.78041648 -0.64612818 -0.55484283
 -0.49675696 -0.44734715 -0.40840359 -0.37591402 -0.34839315 -0.2563414 ]


In [12]:

def ReadPredF(filename):
    ofile_prob_E = open(filename,"r")
    Reference_dict = {}
    idx_jj = 0
    while(True):
        line = ofile_prob_E.readline()
        if not line:
            break
        jj = float(line.split()[1].replace("alpha-1=","").replace(";",""))
        line = ofile_prob_E.readline()
        bin_centers = np.array([float(x) for x in line.split()])

        line = ofile_prob_E.readline()
        line = ofile_prob_E.readline()
        F = np.array([float(x) for x in line.split()])

        line = ofile_prob_E.readline()
        line = ofile_prob_E.readline()
        errF = np.array([float(x) for x in line.split()])

        Reference_dict[jj]=np.stack([bin_centers, F, errF])
    ofile_prob_E.close()
    return Reference_dict

# Conditional flow
- Condition: FM state
- Guidance coefficient $\gamma=1$
## Lattice size = 6x6
$\alpha=0.5, \alpha=1.0, \alpha=9.0$

In [13]:
Pred_dict_withconditions_L6 = []
Pred_dict_withconditions_L36 = []

In [14]:
# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors = prop_cycle.by_key()['color']
colors = [plt.colormaps["gnuplot"](float(i)/4.) for i in range(4)]
print(colors, len(colors))

[(np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0)), (np.float64(0.5009794328681196), np.float64(0.015809545348320027), np.float64(0.9999810273487268), np.float64(1.0)), (np.float64(0.7084919084320762), np.float64(0.12647636278656021), np.float64(0.0), np.float64(1.0)), (np.float64(0.8677218312746247), np.float64(0.4268577244046407), np.float64(0.0), np.float64(1.0))] 4


In [15]:
import os

def read_prediction(dirname, lattice_dim):
    Pred_dict = {}
    Pred_dict["Lattice_dim"] = lattice_dim
    Pred_dict["Flow"] = ReadPredF(os.path.join(dirname, "FES-MAGN.dat"))
    Pred_dict["Flow-PotE"] = ReadPredF(os.path.join(dirname, "FES-E.dat"))
    Pred_dict["FlowProbs-PotE"] = ReadPredF(os.path.join(dirname, "PROB-E.dat"))

    diffstep_list = list(Pred_dict["Flow"].keys())
    return Pred_dict, diffstep_list

In [16]:
def plot_magn(Pred_dict, Reference_dict, Reference_L, Reference_Tlist=[2.4, 1.2], diffstep_list = [18, -1]):
    plt.figure()
    for i, T in enumerate(Reference_Tlist):
        baseline = 0.
        plt.plot(Reference_dict[T][0], Reference_dict[T][1]-baseline, label=r"GT ($k_BT$=%.1f, L=%d)"%(T, Reference_L), c=colors[i], alpha=0.5)
        if T == 1.:
            plt.plot(-Reference_dict[1.][0], Reference_dict[1.][1]-baseline, label=r"GT ($k_BT$=%.1f, L=%d)"%(1., Reference_L), c=colors[i], alpha=0.5)

    for idx_diffstep, diffstep in enumerate(diffstep_list):
        baseline = 0.
        plt.errorbar(Pred_dict["Flow"][list(Pred_dict["Flow"].keys())[diffstep]][0], Pred_dict["Flow"][list(Pred_dict["Flow"].keys())[diffstep]][1]-baseline, yerr=Pred_dict["Flow"][list(Pred_dict["Flow"].keys())[diffstep]][2], label=r"$\alpha$-1=%.1f"%(list(Pred_dict["Flow"].keys())[diffstep]), linestyle='none', c=colors[idx_diffstep], marker="D", ms=3)
        if diffstep == -1:
            plt.errorbar(Pred_dict["Flow"][list(Pred_dict["Flow"].keys())[diffstep]][0], Pred_dict["Flow"][list(Pred_dict["Flow"].keys())[diffstep]][1]-baseline, yerr=Pred_dict["Flow"][list(Pred_dict["Flow"].keys())[diffstep]][2], label=r"$\alpha$-1=%.1f"%(list(Pred_dict["Flow"].keys())[diffstep]), linestyle='none', c=colors[2], marker="D", ms=3)

    setfigform_simple("Magnetization", "Free energy surface ($k_BT$)", ylimit=(None, 6))
    plt.legend(fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))

In [17]:
def plot_potenergy(Pred_dict, Reference_dict, Reference_L, Reference_Tlist=[2.4, 1.2], diffstep_list = [18, -1]):
    plt.figure()
    for i, T in enumerate(Reference_Tlist):
        # if T in [8.0]:
        #     continue
        plt.plot(Reference_dict[T][0]/Reference_L**2, Reference_dict[T][1], label=r"GT ($k_BT$=%.1f, L=%d)"%(T, Reference_L), c=colors[i], alpha=0.5)

    for idx_diffstep, diffstep in enumerate(diffstep_list):
        plt.errorbar(Pred_dict["Flow-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][0]/Reference_L**2, Pred_dict["Flow-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][1], yerr=Pred_dict["Flow-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][2], label=r"$\alpha$-1=%.1f"%(list(Pred_dict["Flow-PotE"].keys())[diffstep]), linestyle='none', c=colors[idx_diffstep], marker="D", ms=3)
        if diffstep == -1:
            plt.errorbar(Pred_dict["Flow-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][0]/Reference_L**2, Pred_dict["Flow-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][1], yerr=Pred_dict["Flow-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][2], label=r"$\alpha$-1=%.1f"%(list(Pred_dict["Flow-PotE"].keys())[diffstep]), linestyle='none', c=colors[2], marker="D", ms=3)

    setfigform_simple("Potential energy (J)", "Negative likelihood")
    plt.legend(fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))

In [18]:
def calc_expectation_potenergy(Pred_dict, diffstep_list):
    Pred_dict["FlowExpectations-PotE"] = []
    for diffstep in range(80):
        Jac_Expectation = Pred_dict["FlowProbs-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][0]
        var_Prob = Pred_dict["FlowProbs-PotE"][list(Pred_dict["Flow-PotE"].keys())[diffstep]][2]**2
        err_Expectation = np.sqrt(Jac_Expectation*var_Prob*Jac_Expectation).sum()
        Expectation = (Pred_dict["FlowProbs-PotE"][diffstep_list[diffstep]][1]*Pred_dict["FlowProbs-PotE"][diffstep_list[diffstep]][0]).sum()
        Pred_dict["FlowExpectations-PotE"].append([diffstep_list[diffstep], Expectation, err_Expectation])

    Pred_dict["FlowExpectations-PotE"] = np.array(Pred_dict["FlowExpectations-PotE"])

In [19]:
def plot_expactation_potenergy(Pred_dict, Reference_dict, Reference_L, Benckmark_T=[1.,], Benckmark_diffstep=[-1]):
    plt.figure(figsize=(13,5))
    plt.subplot(121)
    plt.plot(Reference_dict["PotE"][:,0], Reference_dict["PotE"][:,1]/Reference_L**2, c="k")
    assert len(Benckmark_T) == len(Benckmark_diffstep)
    for i in range(len(Benckmark_T)):
        plt.errorbar(Benckmark_T[i], Pred_dict["FlowExpectations-PotE"][Benckmark_diffstep[i],1]/Reference_L**2, yerr=Pred_dict["FlowExpectations-PotE"][Benckmark_diffstep[i],2]/Reference_L**2, c="blue", fmt="D", ms=8)
    plt.axvline(2/np.log(1+np.sqrt(2)), linestyle="--", c="k")
    setfigform_simple("$kBT$", "$<U>$", xlimit=(None,6))
    plt.ylim((-2.2,-400/36**2))
    plt.subplot(122)
    plt.errorbar(Pred_dict["FlowExpectations-PotE"][:,0], Pred_dict["FlowExpectations-PotE"][:,1]/Reference_L**2, yerr=Pred_dict["FlowExpectations-PotE"][:,2]/Reference_L**2, c="blue", fmt="-")
    setfigform_simple(r"$\alpha$-1", "$<U>$")
    plt.ylim((-2.2,-400/36**2))
    plt.tight_layout()

In [20]:
Pred_dict_c1, difftime_list = read_prediction("val_baseline_latt6x6/epoch159_IntStep80_AMax10_clspd1.0_scoreG1.0", (6,6))

FileNotFoundError: [Errno 2] No such file or directory: 'val_baseline_latt6x6/epoch159_IntStep80_AMax10_clspd1.0_scoreG1.0/FES-MAGN.dat'

In [ ]:
calc_expectation_potenergy(Pred_dict_c1, difftime_list)

In [ ]:
plot_expactation_potenergy(Pred_dict_c1, Expectation_Reference_dict_latt6, 6)

In [ ]:

Pred_dict_ = {}
Pred_dict_["guidance_class"] = "pd1.0"
Pred_dict_["shuffle_freq"] = 0.0
Pred_dict_["guidance_coeff"] = 1.0
Pred_dict_["Predictions"] = Pred_dict_c1
Pred_dict_withconditions_L6.append(Pred_dict_)

In [ ]:

Pred_dict_c2, difftime_list = read_prediction("val_baseline_latt6x6/epoch159_IntStep80_AMax10_clsT2.4L6shuffle0.6_scoreG1.0_bs10240", (6,6))

In [ ]:
calc_expectation_potenergy(Pred_dict_c2, difftime_list)

In [ ]:
plot_magn(Pred_dict_c2, Reference_dict_latt6, 6, Reference_Tlist=[3.2, 2.8, 2.4])

In [ ]:
plot_potenergy(Pred_dict_c2, Reference_dict_potE_latt6, 6, Reference_Tlist=[3.2, 2.8, 2.4])

In [ ]:
plot_expactation_potenergy(Pred_dict_c2, Expectation_Reference_dict_latt6, 6, Benckmark_T=[2.4], Benckmark_diffstep=[-1])

In [ ]:

Pred_dict_ = {}
Pred_dict_["guidance_class"] = "T2.4L6"
Pred_dict_["shuffle_freq"] = 0.6
Pred_dict_["guidance_coeff"] = 1.0
Pred_dict_["Predictions"] = Pred_dict_c2
Pred_dict_withconditions_L6.append(Pred_dict_)

In [ ]:

Pred_dict_c2_c, difftime_list = read_prediction("logs-dir-ising/latt6x6T3.2/kernel3x3_timeembed_symmetrized/clsfreeG/val_baseline_latt6x6/epoch159_IntStep80_AMax10_clsT2.4L6shuffle0.55_scoreG1.0_bs50000", (6,6))
calc_expectation_potenergy(Pred_dict_c2_c, difftime_list)
plot_magn(Pred_dict_c2_c, Reference_dict_potE_latt6, 6, Reference_Tlist=[3.2, 2.8, 2.4])
plot_potenergy(Pred_dict_c2_c, Reference_dict_potE_latt6, 6, Reference_Tlist=[3.2, 2.8, 2.4])

In [ ]:
plot_expactation_potenergy(Pred_dict_c2_c, Expectation_Reference_dict_latt6, 6, Benckmark_T=[2.4], Benckmark_diffstep=[-1])

## Lattice size = 12x12

In [21]:
ref_dirname = "/nfs/scistore14/chenggrp/ptuo/NeuralRG/data/ising-latt%dx%d-T4.0/latt%dx%d/"%(12,12,12,12)
Reference_dict_latt12 = ReadReferenceF(os.path.join(ref_dirname, "F-MAGN-REF.dat"))
Reference_dict_potE_latt12 = ReadReferenceF_potE(os.path.join(ref_dirname, "F-E-REF.dat"))
Expectation_Reference_dict_latt12 = {}
Expectation_Reference_dict_latt12["PotE"] = np.loadtxt(os.path.join(ref_dirname, "Expectation-E-REF.dat"), skiprows=1)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Lattice size = 36x36

In [ ]:
ref_dirname = "/nfs/scistore14/chenggrp/ptuo/NeuralRG/data/ising-latt%dx%d-T4.0/latt%dx%d/"%(36,36,36,36)
Reference_dict_latt36 = ReadReferenceF(os.path.join(ref_dirname, "F-MAGN-REF.dat"))
Reference_dict_potE_latt36 = ReadReferenceF_potE(os.path.join(ref_dirname, "F-E-REF.dat"))


In [ ]:
Expectation_Reference_dict_latt36 = {}
Expectation_Reference_dict_latt36["PotE"] = np.loadtxt(os.path.join(ref_dirname, "Expectation-E-REF.dat"), skiprows=1)

In [ ]:
Pred_dict_c1_latt36, difftime_list = read_prediction("logs-dir-ising/latt6x6T3.2/kernel3x3_timeembed_symmetrized/clsfreeG/val_baseline_latt36x36/epoch159_IntStep80_AMax10_clspd1.0_scoreG1.0_testsymm1", (36,36))

In [ ]:
calc_expectation_potenergy(Pred_dict_c1_latt36, difftime_list)

In [ ]:
plot_expactation_potenergy(Pred_dict_c1_latt36, Expectation_Reference_dict_latt36, 36)

In [ ]:

Pred_dict_ = {}
Pred_dict_["guidance_class"] = "pd1.0"
Pred_dict_["shuffle_freq"] = 0.0
Pred_dict_["guidance_coeff"] = 1.0
Pred_dict_["Predictions"] = Pred_dict_c1_latt36
Pred_dict_withconditions_L36.append(Pred_dict_)